In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

# Cargar el dataset
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_2/DATOS_PRPROCESADOS_2_SIN_COVID.csv')

# Eliminar filas que contengan "-999" o -999 en cualquier columna
df = df[~df.isin(["-999", -999]).any(axis=1)]

# Eliminar la columna identificadora
df.drop("gidenpac", axis=1, inplace=True)

# Lista de columnas que NO se quieren convertir a categoría (se mantendrán como numéricas)
numeric_features = ['ds_edad', 'ds_estancia', 'ds_pre_oper', 'ds_post_oper', 'ds_vivo_alta']

# Convertir todas las columnas que NO estén en numeric_features a tipo 'category'
for col in df.columns:
    if col not in numeric_features:
        df[col] = df[col].astype('category')

# Definir las features y la variable objetivo.
# Se eliminan las columnas que no se usarán como predictores.
X = df.drop(["gsitalta", "ds_vivo_alta", "ds_estancia", "ds_post_oper", "ds_pre_oper"], axis=1)
y = df["ds_pre_oper"]

# Identificar columnas numéricas y categóricas
num_cols = X.select_dtypes(include=[np.number]).columns
cat_cols = X.select_dtypes(include=["object", "category"]).columns

# Crear un preprocesador que escala las variables numéricas y aplica OneHotEncoder a las categóricas
preprocessor = ColumnTransformer(transformers=[
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
])

# Definir un pipeline que incluya el preprocesamiento y el modelo XGBoost
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", XGBRegressor(random_state=42, objective="reg:squarederror"))
])

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir una búsqueda de hiperparámetros para XGBoost
param_grid = {
    "regressor__n_estimators": [100, 200, 300],
    "regressor__max_depth": [3, 5, 7],
    "regressor__learning_rate": [0.01, 0.1, 0.2],
    "regressor__subsample": [0.8, 1.0]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring="neg_mean_squared_error", n_jobs=-1)
grid_search.fit(X_train, y_train)

# Realizar predicciones y evaluar el modelo
y_pred = grid_search.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
explained_var = explained_variance_score(y_test, y_pred)

print("\nError cuadrático medio (MSE):", mse)
print("Error absoluto medio (MAE):", mae)
print("R² (coeficiente de determinación):", r2)
print("Varianza explicada:", explained_var)

# Verificar las columnas generadas tras el preprocesamiento
preprocessor_final = grid_search.best_estimator_.named_steps["preprocessor"]

# Obtener los nombres de las columnas resultantes (requiere scikit-learn >= 1.0)
try:
    feature_names = preprocessor_final.get_feature_names_out()
    print("\nCaracterísticas utilizadas por el modelo:")
    print(feature_names)
except AttributeError:
    print("\nLa versión de scikit-learn no soporta 'get_feature_names_out()'.")


FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_2/DATOS_PREPROCESADOS_2_SIN_COVID.csv'

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

# Cargar los datos
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_2/DATOS_PREPROCESADOS_2.csv')

# Eliminar filas con "-999" o -999
df = df[~df.isin(["-999", -999]).any(axis=1)]
df.drop("gidenpac", axis=1, inplace=True)

# Lista de variables categóricas a convertir
cat_features = [
    'gsitalta', 'itipsexo', 'itipingr', 'ireingre', 'iotrocen', 'gdiagalt',
    'ds_izq_der', 'ds_turno', 'ds_dia_semana_llegada', 'ds_mes_llegada',
    'ds_centro_afueras', 'ds_alergia_medicamentosa', 
    'ds_alergia_alimenticia', 'ds_otras_alergias', 'ds_ITU', 'ds_anemia',
    'ds_vitamina_d', 'ds_insuficiencia_respiratoria', 'ds_insuficiencia_cardiaca',
    'ds_deterioro_cognitivo', 'ds_insuficiencia_renal', 'ds_HTA', 'ds_diabetes'
]

# Convertir estas variables a tipo 'category'
for col in cat_features:
    if col in df.columns:
        df[col] = df[col].astype("category")

# Definir X e y. Se eliminan columnas que no se usarán como predictores
X = df.drop(["gsitalta", "ds_vivo_alta", "ds_estancia", "ds_post_oper", "ds_pre_oper"], axis=1)
y = df["ds_pre_oper"]

# Identificar columnas numéricas y categóricas
num_cols = X.select_dtypes(include=[np.number]).columns
cat_cols = X.select_dtypes(include=["object", "category"]).columns

#print("Variables categóricas que se utilizarán:", list(cat_cols))

# Definir un preprocesador que escale las numéricas y codifique las categóricas
preprocessor = ColumnTransformer(transformers=[
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
])

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir una lista de modelos a comparar
models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(random_state=42),
    "Lasso": Lasso(random_state=42),
    "RandomForest": RandomForestRegressor(random_state=42),
    "GradientBoosting": GradientBoostingRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42, objective="reg:squarederror")
}

results = []

for name, model in models.items():
    # Crear un pipeline para cada modelo
    pipeline = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("regressor", model)
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    ev = explained_variance_score(y_test, y_pred)
    
    results.append({
        "Model": name,
        "MSE": mse,
        "MAE": mae,
        "R2": r2,
        "Explained Variance": ev
    })
    
    print(f"\n--- {name} ---")
    print("MSE:", mse)
    print("MAE:", mae)
    print("R²:", r2)
    print("Varianza explicada:", ev)

# Convertir los resultados a DataFrame para compararlos fácilmente
results_df = pd.DataFrame(results)
print("\nComparación de modelos:")
print(results_df)

# Opcional: Si deseas ver las características finales utilizadas por el preprocesador en alguno de los pipelines,
# puedes usar get_feature_names_out() (requiere scikit-learn >= 1.0)
preprocessor_final = Pipeline(steps=[("preprocessor", preprocessor)]).fit(X_train)
try:
    feature_names = preprocessor_final.named_steps["preprocessor"].get_feature_names_out()
    #print("\nCaracterísticas utilizadas por el modelo:")
    #print(feature_names)
except AttributeError:
    print("\nLa versión de scikit-learn no soporta 'get_feature_names_out()'.")



--- LinearRegression ---
MSE: 1206.8461490385168
MAE: 23.776677476191157
R²: -347.6354965229197
Varianza explicada: -347.63432505811915

--- Ridge ---
MSE: 6.542570780295813
MAE: 1.777521221067775
R²: -0.8900275021319053
Varianza explicada: -0.8111131229396882

--- Lasso ---
MSE: 3.620568689739013
MAE: 1.4224449556358856
R²: -0.04591522610245513
Varianza explicada: 0.0

--- RandomForest ---
MSE: 4.355157541899442
MAE: 1.2616759776536315
R²: -0.25812433777521315
Varianza explicada: -0.2566239861514328

--- GradientBoosting ---
MSE: 5.370699122768715
MAE: 1.4260873621397816
R²: -0.551495488329989
Varianza explicada: -0.5148577059145594

--- XGBoost ---
MSE: 6.011774749622959
MAE: 1.4647816612317575
R²: -0.7366902828216553
Varianza explicada: -0.7030901922635324

Comparación de modelos:
              Model          MSE        MAE          R2  Explained Variance
0  LinearRegression  1206.846149  23.776677 -347.635497         -347.634325
1             Ridge     6.542571   1.777521   -0.890

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.feature_selection import RFE
from sklearn.linear_model import Lasso, LassoCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

# Cargamos el dataset
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_2/DATOS_PREPROCESADOS_2.csv')

# Eliminar filas que contengan "-999" o -999 en cualquier columna
df = df[~df.isin(["-999", -999]).any(axis=1)]

# Eliminamos la columna identificadora
df.drop("gidenpac", axis=1, inplace=True)

# Lista de variables categóricas a tratar como tales
cat_features = [
    'gsitalta', 'itipsexo', 'itipingr', 'ireingre', 'iotrocen', 'gdiagalt',
    'ds_izq_der', 'ds_turno', 'ds_dia_semana_llegada', 'ds_mes_llegada',
    'ds_centro_afueras', 'ds_alergia_medicamentosa', 'movilidad', 'riesgo_caida',
    'ds_alergia_alimenticia', 'ds_otras_alergias', 'ds_ITU', 'ds_anemia',
    'ds_vitamina_d', 'ds_insuficiencia_respiratoria', 'ds_insuficiencia_cardiaca',
    'ds_deterioro_cognitivo', 'ds_insuficiencia_renal', 'ds_HTA', 'ds_diabetes'
]

# Convertir las variables de la lista a tipo 'category'
for col in cat_features:
    if col in df.columns:
        df[col] = df[col].astype("category")

# Verificar los tipos de datos
print("Tipos de datos después de la conversión:")
print(df.dtypes)

# Definir X e y.
# Se eliminan las columnas que no se utilizarán como predictores
X = df.drop(["gsitalta", "ds_vivo_alta", "ds_estancia", "ds_post_oper", "ds_pre_oper"], axis=1)
y = df["ds_pre_oper"]

# Identificar columnas numéricas y categóricas
num_cols = X.select_dtypes(include=[np.number]).columns
cat_cols = X.select_dtypes(include=["object", "category"]).columns
print("\nVariables categóricas que se utilizarán:", list(cat_cols))

# Preprocesador: uso de RobustScaler para las variables numéricas y OneHotEncoder para las categóricas
preprocessor = ColumnTransformer(transformers=[
    ("num", RobustScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
])

# Usaremos Lasso con validación cruzada para la selección de características.
# Esta será la base para el RFE y también el modelo final.
# Se puede optimizar el parámetro alpha.
base_estimator = LassoCV(cv=5, random_state=42)

# Definir RFE. El número de features a seleccionar se optimizará.
rfe = RFE(estimator=base_estimator, n_features_to_select=30)

# Modelo final: Usamos Lasso (puede ser el mismo que en la selección)
regressor = Lasso(random_state=42)

# Pipeline: preprocesador -> selección de características (RFE) -> regresor
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("feature_selection", rfe),
    ("regressor", regressor)
])

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el espacio de búsqueda de hiperparámetros para optimizar:
# - El número de features a seleccionar en RFE
# - El parámetro de regularización de Lasso final (alpha)
param_grid = {
    "feature_selection__n_features_to_select": [20, 30, 40, 50],
    "regressor__alpha": [0.001, 0.01, 0.1, 1, 10]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring="neg_mean_squared_error", n_jobs=-1)
grid_search.fit(X_train, y_train)

print("\nMejores parámetros encontrados:")
print(grid_search.best_params_)

# Predicciones y evaluación
y_pred = grid_search.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
explained_var = explained_variance_score(y_test, y_pred)

print("\nError cuadrático medio (MSE):", mse)
print("Error absoluto medio (MAE):", mae)
print("R² (coeficiente de determinación):", r2)
print("Varianza explicada:", explained_var)

# Verificar las características utilizadas por el modelo
preprocessor_final = grid_search.best_estimator_.named_steps["preprocessor"]
try:
    feature_names = preprocessor_final.get_feature_names_out()
    print("\nCaracterísticas utilizadas tras el preprocesamiento:")
    print(feature_names)
except AttributeError:
    print("\nLa versión de scikit-learn no soporta 'get_feature_names_out()'.")


Tipos de datos después de la conversión:
itipsexo              category
fllegada                object
gsitalta              category
iotrocen              category
gdiagalt              category
                        ...   
gfarmaco_P660685.8       int64
gfarmaco_P660686.5       int64
gfarmaco_P667865         int64
ds_comorbilidades        int64
ds_ulcera_presion        int64
Length: 146, dtype: object

Variables categóricas que se utilizarán: ['itipsexo', 'fllegada', 'iotrocen', 'gdiagalt', 'ds_izq_der', 'ds_turno', 'ds_dia_semana_llegada', 'ds_mes_llegada', 'ds_centro_afueras', 'ds_alergia_medicamentosa', 'ds_alergia_alimenticia', 'ds_otras_alergias', 'ds_ITU', 'ds_anemia', 'ds_vitamina_d', 'ds_insuficiencia_respiratoria', 'ds_insuficiencia_cardiaca', 'ds_deterioro_cognitivo', 'ds_insuficiencia_renal', 'ds_HTA', 'ds_diabetes']


/home/ubuntu/STG-fractura_cadera/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.8691975692472198, tolerance: 0.34237549234135667
  model = cd_fast.sparse_enet_coordinate_descent(
/home/ubuntu/STG-fractura_cadera/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.0058075926319532, tolerance: 0.34237549234135667
  model = cd_fast.sparse_enet_coordinate_descent(
/home/ubuntu/STG-fractura_cadera/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7898989050558782, tolerance: 0.34237549234135667
  model = cd_fast.sparse_enet_coordinate_descent(
/hom

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
# Cargamos el dataset
df = pd.read_csv('/home/ubuntu/STG-fractura_cadera/2025_02/models/PREPROCESADO_2/DATOS_PREPROCESADOS_2.csv')

# Eliminar filas que contengan "-999" o -999 en cualquier columna
df = df[~df.isin(["-999", -999]).any(axis=1)]

# Eliminamos la columna identificadora
df.drop("gidenpac", axis=1, inplace=True)
# Supongamos que 'df' es tu DataFrame original.
# Eliminamos columnas que no se usarán para el modelado (por ejemplo, identificadores o variables post-operatorias)
df_model = df.drop(columns=['gsitalta', 'ds_vivo_alta','ds_post_oper', 'ds_vivo_alta'])

# Separamos la variable objetivo y las variables predictoras
X = df_model.drop(columns=['ds_pre_oper'])
y = df_model['ds_pre_oper']

# Identificamos columnas numéricas y categóricas
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['category', 'object']).columns.tolist()

# Dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocesamiento: escalamos las variables numéricas y codificamos las categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
    ])

# Creamos un pipeline que integra el preprocesamiento y el modelo de regresión
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(random_state=42))
])

# Definimos la grilla de hiperparámetros para la búsqueda
param_grid = {
    'regressor__learning_rate': [0.01, 0.1],
    'regressor__max_depth': [3, 5],
    'regressor__n_estimators': [100, 200]
}

# Configuramos GridSearchCV para optimizar el MSE (se usa la métrica negativa de MSE)
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Imprimimos los mejores parámetros encontrados
print("Mejores parámetros encontrados:", grid_search.best_params_)

# Utilizamos el mejor modelo para predecir en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Calculamos las métricas de evaluación adicionales
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
explained_variance = explained_variance_score(y_test, y_pred)

print("MSE:", mse)
print("MAE:", mae)
print("R²:", r2)
print("Varianza explicada:", explained_variance)


Mejores parámetros encontrados: {'regressor__learning_rate': 0.01, 'regressor__max_depth': 3, 'regressor__n_estimators': 200}
MSE: 4.162088587314377
MAE: 1.3668103643842067
R²: -0.2023502932554948
Varianza explicada: -0.15465840832103184
